In [1]:
import numpy as np
import pandas as pd
import math

import torch
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from pytorch_lightning import seed_everything
seed_everything(42, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)


import dcor

from datetime import datetime

from torch import nn

import argparse

import os 



[rank: 0] Global seed set to 42


In [2]:
parser = argparse.ArgumentParser(description="Running BENN")
parser.add_argument('--model1', default=2, type = int, help = 'model1')
parser.add_argument('--model2', default=1, type = int, help = 'model2')
parser.add_argument('--n', default=5000, type = int, help = 'n')
parser.add_argument('--m', default=1, type = int, help = 'm')
parser.add_argument('--l1', default=4, type = int, help = 'l1')
parser.add_argument('--l2', default=4, type = int, help = 'l2')
parser.add_argument('--r1', default=300, type = int, help = 'r1')
parser.add_argument('--r2', default=300, type = int, help = 'r2')
parser.add_argument('--d', default=1, type = int, help = 'd')
parser.add_argument('--t', default=1, type = int, help = 't')
args = parser.parse_args([])
model1 = args.model1
model2 = args.model2
n = args.n
m = args.m
l1 = args.l1
l2 = args.l2
r1 = args.r1
r2 = args.r2
res_d = args.d
t = args.t
print(model1, model2, n, m, l1, l2, r1, r2, res_d, t)

2 1 5000 1 4 4 300 300 1 1


In [3]:
# Create device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [6]:
for model1 in range(1,4):
    for model2 in range(1,4):
        dcor_list=[]
        for t in range(1,3):
            x_train=pd.read_csv("./data-prelim/model" + str(model1) + "-" + str(model2) + "-" + str(n) + "/x_train_" + str(t) + ".csv")
            x_train=x_train.drop('Unnamed: 0', axis=1)
            y_train=pd.read_csv("./data-prelim/model" + str(model1) + "-" + str(model2) + "-" + str(n) + "/y_train_" + str(t) + ".csv")
            y_train=y_train.drop('Unnamed: 0', axis=1)
            x_test=pd.read_csv("./data-prelim/model" + str(model1) + "-" + str(model2) + "-" + str(n) + "/x_test_" + str(t) + ".csv")
            x_test=x_test.drop('Unnamed: 0', axis=1)
            y_test=pd.read_csv("./data-prelim/model" + str(model1) + "-" + str(model2) + "-" + str(n) + "/y_test_" + str(t) + ".csv")
            y_test=y_test.drop('Unnamed: 0', axis=1)
            z_test=pd.read_csv("./data-prelim/model" + str(model1) + "-" + str(model2) + "-" + str(n) + "/z_test_" + str(t) + ".csv")
            z_test=z_test.drop('Unnamed: 0', axis=1)

            n=x_train.shape[0]
            p=x_train.shape[1]


            x_train = torch.tensor(x_train.values).to(torch.float)
            x_test = torch.tensor(x_test.values).to(torch.float)
            if m==1:
                y_train = torch.tensor(y_train.values).to(torch.float)
                y_test = torch.tensor(y_test.values).to(torch.float)
            else:
                y_trans_train = (y_train - y_train.mean()) / y_train.std()
                y_trans_test = (y_test - y_train.mean()) / y_train.std()
                for i in range(2,m+1):
                    y_train_intermediate=y_train**i/math.factorial(i)
                    y_test_intermediate=y_test**i/math.factorial(i)
                    y_test_intermediate = (y_test_intermediate - y_train_intermediate.mean()) / y_train_intermediate.std()
                    y_train_intermediate = (y_train_intermediate - y_train_intermediate.mean()) / y_train_intermediate.std()
                    y_trans_train = np.concatenate((y_trans_train,y_train_intermediate), axis=1)
                    y_trans_test = np.concatenate((y_trans_test,y_test_intermediate), axis=1)
                y_train = torch.tensor(y_trans_train).to(torch.float)
                y_test = torch.tensor(y_trans_test).to(torch.float)


            mse_loss = nn.MSELoss()
            class nn_dr_reg_model(nn.Module):
                def __init__(self, input_features, output_features, dim_red_features, hidden_units_d, hidden_units_e, dim_red_layers, ens_reg_layers):
                    super().__init__()
                    model_dim_red=[]
                    model_dim_red.append(nn.Linear(in_features=input_features, 
                                                out_features=hidden_units_d))
                    model_dim_red.append(nn.ReLU())
                    for i in range(1,dim_red_layers):
                        model_dim_red.append(nn.Linear(in_features=hidden_units_d, 
                                                    out_features=hidden_units_d))
                        model_dim_red.append(nn.ReLU())
                    model_dim_red.append(nn.Linear(in_features=hidden_units_d, 
                                                out_features=dim_red_features))
                    self.dim_red_layer_stack = nn.Sequential(*model_dim_red)

                    model_ens_reg=[]
                    model_ens_reg.append(nn.Linear(in_features=dim_red_features, out_features=hidden_units_e))
                    model_ens_reg.append(nn.ReLU())
                    for i in range(1,ens_reg_layers):
                        model_ens_reg.append(nn.Linear(in_features=hidden_units_e, out_features=hidden_units_e))
                        model_ens_reg.append(nn.ReLU())
                    model_ens_reg.append(nn.Linear(in_features=hidden_units_e, out_features=output_features))
                    self.ens_reg_layer_stack = nn.Sequential(*model_ens_reg)

                def forward(self, x):
                    suff_predictor = self.dim_red_layer_stack(x)
                    ens_output = self.ens_reg_layer_stack(suff_predictor)
                    return ens_output, suff_predictor


            model_nn = nn_dr_reg_model(input_features=p, 
                                    output_features=m, 
                                    dim_red_features=res_d, 
                                    hidden_units_d=r1,
                                    hidden_units_e=r2,
                                    dim_red_layers=l1, 
                                    ens_reg_layers=l2
                                    ).to(device)
            model_nn
            optimizer = torch.optim.Adam(model_nn.parameters(), 
                                        lr=0.001)
            epochs = 200
            x_train, y_train = x_train.to(device), y_train.to(device)
            x_test = x_test.to(device)
            for epoch in range(epochs):
                ### Training
                model_nn.train()
                y_pred_train, y_suff_train = model_nn(x_train) 
                loss = mse_loss(y_pred_train, y_train) 
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                ### Testing
                model_nn.eval()
                y_pred_test, y_suff_test = model_nn(x_test)
                loss_test = mse_loss(y_pred_test, y_test) 
                dcor_test = dcor.distance_correlation(np.float64(y_suff_test.detach().numpy()),np.float64(z_test))

                if epoch % 25 == 0:
                    print(f"Epoch: {epoch} | Loss: {loss:.5f} | Test Loss: {loss_test:.5f}")
                    now = datetime.now()
                    current_time = now.strftime("%H:%M:%S")
                    print("Current Time =", current_time)
            model_nn.eval()
            with torch.inference_mode():
                y_pred_test, y_suff_test = model_nn(x_test)
            y_suff_test=y_suff_test.numpy()
            if res_d==1:
                dcor_current=dcor.distance_correlation(np.float64(y_suff_test),np.float64(z_test.to_numpy()),method="naive")
            elif res_d==2:
                dcor_current=[dcor.distance_correlation(np.float64(y_suff_test),np.float64(z_test.to_numpy()),method="naive"),
                              dcor.distance_correlation(np.float64(y_suff_test),np.float64(z_test.to_numpy()[:,0]),method="naive"),
                              dcor.distance_correlation(np.float64(y_suff_test),np.float64(z_test.to_numpy()[:,1]),method="naive")]
            dcor_list.append(dcor_current)
            print(model1, model2, t, dcor_current)
        dcor_list_df=pd.DataFrame(dcor_list)
        #dcor_list_df.to_csv("./results-BENN-unified-std/result-" + str(model1) + "-" + str(model2) + "-" + str(m) + "-" + str(n) + ".csv")
        print(model1, model2, np.mean(dcor_list), np.std(dcor_list))

Epoch: 0 | Loss: 4.88627 | Test Loss: 4.46475
Current Time = 12:38:25
Epoch: 25 | Loss: 1.55305 | Test Loss: 1.34544
Current Time = 12:38:26
Epoch: 50 | Loss: 0.39593 | Test Loss: 0.51795
Current Time = 12:38:28
Epoch: 75 | Loss: 0.25271 | Test Loss: 0.40891
Current Time = 12:38:29
Epoch: 100 | Loss: 0.14702 | Test Loss: 0.37591
Current Time = 12:38:30
Epoch: 125 | Loss: 0.09483 | Test Loss: 0.41011
Current Time = 12:38:31
Epoch: 150 | Loss: 0.06051 | Test Loss: 0.44334
Current Time = 12:38:32
Epoch: 175 | Loss: 0.05313 | Test Loss: 0.48240
Current Time = 12:38:33
1 1 1 0.9456371261821748
Epoch: 0 | Loss: 5.30345 | Test Loss: 4.72156
Current Time = 12:38:35
Epoch: 25 | Loss: 2.91308 | Test Loss: 2.92049
Current Time = 12:38:36
Epoch: 50 | Loss: 2.68975 | Test Loss: 2.60451
Current Time = 12:38:37
Epoch: 75 | Loss: 0.45436 | Test Loss: 0.61924
Current Time = 12:38:38
Epoch: 100 | Loss: 0.33057 | Test Loss: 0.45967
Current Time = 12:38:39
Epoch: 125 | Loss: 0.19868 | Test Loss: 0.36170
C